Тестовое задание:
С использованием методов process mining написать алгоритм сбора статистики по логу и определить-
1. Количество всех возможных путей процесса 
2. Основной путь процесса 
3. Этапы, удлиняющие основной путь
4. Длительность этапов процесса

**Импорт необходимых зависимостей.**


In [73]:
from sberpm import DataHolder
import pandas as pd

In [74]:
path = 'static\log.csv'

In [75]:
df = pd.read_csv(path)

In [76]:
df['Дата'] = df.apply(lambda row: str(row['Дата']) + ' ' + str(row['Часовой пояс']), axis=1)
df

,Ид,Дата,Часовой пояс,Операция
0,10010.0,03.10.2022 2:16 UTC+0,UTC+0,Блокировка
1,10030.0,03.10.2022 2:20 UTC+0,UTC+0,Блокировка
2,10050.0,03.10.2022 4:20 UTC+0,UTC+0,Блокировка
3,10060.0,03.10.2022 4:50 UTC+0,UTC+0,Блокировка
4,10070.0,03.10.2022 5:01 UTC+0,UTC+0,Блокировка
...,...,...,...,...
67778,13295.0,12.10.2022 13:29 UTC+5,UTC+5,Регистрация
67779,13305.0,12.10.2022 13:42 UTC+5,UTC+5,Регистрация
67780,13315.0,12.10.2022 13:58 UTC+5,UTC+5,Регистрация
67781,13325.0,12.10.2022 14:15 UTC+5,UTC+5,Регистрация


In [77]:
data_holder = DataHolder(data=df, 
                         id_column='Ид', 
                         activity_column='Операция', 
                         start_timestamp_column='Дата',
                         text_column='Часовой пояс',
                         yearfirst= True)

In [78]:
data_holder.data.head()

,Ид,Дата,Часовой пояс,Операция
0,10001.0,2022-01-10 01:00:00+00:00,UTC+1,Прием взыскание
1,10001.0,2022-01-10 01:00:00+00:00,UTC+1,Прием взыскание
2,10001.0,2022-01-10 01:00:00+00:00,UTC+1,Прием взыскание
3,10001.0,2022-01-10 01:00:00+00:00,UTC+1,Прием взыскание
4,10001.0,2022-01-10 07:58:00+00:00,UTC+1,Регистрация


**Отрисовка всех ребер без фильтрации**

In [79]:
from sberpm.miners import SimpleMiner

In [80]:
simple_miner = SimpleMiner(data_holder)
simple_miner.apply()
graph = simple_miner.graph

In [81]:
graph

Nodes: ['Прием взыскание', 'Регистрация', 'Блокировка', 'Назначено в рабочую группу', 'Исполнено', 'Проверка_1', 'Проверка_2', 'Ожидание', 'Исполнeно', 'Прием арест', 'Проверка_3', 'startevent', 'endevent']
Edges: [('startevent', 'Прием взыскание'), ('startevent', 'Прием арест'), ('startevent', 'Исполнeно'), ('startevent', 'Исполнено'), ('startevent', 'Назначено в рабочую группу'), ('startevent', 'Проверка_2'), ('Исполнено', 'endevent'), ('Ожидание', 'endevent'), ('Исполнeно', 'endevent'), ('Назначено в рабочую группу', 'endevent'), ('Блокировка', 'endevent'), ('Проверка_1', 'endevent'), ('Ожидание', 'Ожидание'), ('Исполнeно', 'Исполнeно'), ('Проверка_1', 'Проверка_2'), ('Назначено в рабочую группу', 'Исполнено'), ('Регистрация', 'Проверка_2'), ('Проверка_2', 'Проверка_2'), ('Блокировка', 'Ожидание'), ('Исполнено', 'Прием взыскание'), ('Блокировка', 'Назначено в рабочую группу'), ('Прием взыскание', 'Проверка_1'), ('Регистрация', 'Блокировка'), ('Прием арест', 'Регистрация'), ('Прием в

In [82]:
%matplotlib inline
from sberpm.visual import GraphvizPainter


In [83]:
printer = GraphvizPainter()
printer.apply(graph)
printer.show()

In [84]:
from sberpm.miners import CausalMiner

In [85]:
causal_miner = CausalMiner(data_holder)
causal_miner.apply()
graph = causal_miner.graph

painter = GraphvizPainter()
painter.apply(graph)
painter.show()

**1. Колличество путей процесса**

In [86]:
all_ways = list(graph.get_edges())
all_ways_count = len(all_ways)
print(f'Колличество всех возможных путей в процессе: {all_ways_count}')

Колличество всех возможных путей в процессе: 34


**2. Основной путь процесса**

Осмелюсь предположить что основной путь процесса скрывается за цепочкой с самым длинным исполнением.

In [87]:
from sberpm.metrics import TransitionMetric 

все узлы процесса (Для casual без петлей)

In [88]:
graph.get_nodes()

[Node: Прием взыскание,
 Node: Регистрация,
 Node: Блокировка,
 Node: Назначено в рабочую группу,
 Node: Исполнено,
 Node: Проверка_1,
 Node: Проверка_2,
 Node: Ожидание,
 Node: Исполнeно,
 Node: Прием арест,
 Node: Проверка_3,
 Node: startevent,
 Node: endevent]

In [89]:
from sberpm.metrics import ActivityMetric

In [90]:
activity_metric = ActivityMetric(data_holder, time_unit='d')
activity_metric.max_duration()

Операция
Блокировка                    661.822917
Исполнeно                     332.454861
Исполнено                     301.704167
Назначено в рабочую группу    661.906944
Ожидание                        0.000000
Прием арест                     0.646528
Прием взыскание                 0.978472
Проверка_1                    386.456944
Проверка_2                     91.817361
Проверка_3                    152.707639
Регистрация                    60.963194
Name: max_duration, dtype: float64

In [91]:
from sberpm.metrics import TraceMetric

Ищем цепочку с максимально долгой продолжительностью

In [92]:
trace_metric = TraceMetric(data_holder, time_unit='d')

total_d = trace_metric.total_duration()
max(total_d)

442131.0

In [93]:
total_d = total_d.to_dict()
max(total_d)

('Проверка_2',
 'Проверка_2',
 'Проверка_2',
 'Проверка_2',
 'Назначено в рабочую группу',
 'Назначено в рабочую группу',
 'Назначено в рабочую группу',
 'Назначено в рабочую группу',
 'Блокировка',
 'Блокировка',
 'Блокировка',
 'Блокировка',
 'Прием взыскание',
 'Прием взыскание',
 'Прием взыскание',
 'Прием взыскание',
 'Регистрация',
 'Регистрация',
 'Регистрация',
 'Регистрация',
 'Проверка_1',
 'Проверка_1',
 'Проверка_1',
 'Проверка_1',
 'Ожидание',
 'Ожидание',
 'Ожидание',
 'Ожидание')

**3. Этапы, удлиняющие основной путь**

Смею предположить что увеличение времени основного процесса кроется в операциях 'Блокировка', 'Отправка в общую группу'. А так же возникают вопросы к петлям внутр графа. Возможно это возникает по причине того что я неправильно использовал поля часовой пояс.


**4. Длительность этапов процесса**

In [94]:
transition_metric = TransitionMetric(data_holder, time_unit='d')
transition_metric.total_duration()

transition
(Блокировка, Блокировка)                                         0.000000
(Блокировка, Исполнeно)                                       8395.687500
(Блокировка, Исполнено)                                      15082.461806
(Блокировка, Назначено в рабочую группу)                         0.000000
(Блокировка, Ожидание)                                       62543.340278
(Блокировка, Прием арест)                                       56.350694
(Блокировка, Прием взыскание)                                   57.593056
(Исполнeно, Исполнeно)                                           0.000000
(Исполнeно, Прием взыскание)                                 11792.913194
(Исполнено, Исполнено)                                           0.000000
(Исполнено, Прием взыскание)                                 16000.336806
(Назначено в рабочую группу, Блокировка)                         2.761806
(Назначено в рабочую группу, Исполнeно)                      68059.359722
(Назначено в рабочую группу